In [26]:
import pandas as pd
import numpy as np
from catboost import CatBoostRegressor
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split

In [27]:
ybas = pd.read_csv('./data/res_baseline.csv', sep=",", index_col=0)
ybas = ybas['regression']
ybas

0      -0.000255
1       0.002552
2       0.000964
3       0.000405
4      -0.001619
          ...   
8941   -0.000073
8942    0.002533
8943    0.005476
8944    0.004484
8945   -0.001472
Name: regression, Length: 8946, dtype: float64

In [28]:
data = pd.read_csv('processed_data.csv', sep=",")

In [29]:
data

,id_series,Index Name,Index - 9,Index - 8,Index - 7,Index - 6,Index - 5,Index - 4,Index - 3,Index - 2,...,list_speakers_ecb,list_speakers_fed,V_Cible_categ,polarity_score_ecb_LM,polarity_score_fed_LM,polarity_fed_LM,polarity_ecb_LM,sentiment_label_ecb_LM,sentiment_label_fed_LM,sentiment_label_LM
0,0,V2X Index,0.001045,0.005841,0.003832,-0.027519,-0.103565,-0.045086,-0.011265,0.005164,...,"['Philip R. Lane', 'Luis de Guindos']",['Vice Chairman Richard H. Clarida'],Hausse,"{'Positive': 81, 'Negative': 95, 'Polarity': -...","{'Positive': 57, 'Negative': 84, 'Polarity': -...",-0.191489,-0.079545,neutral,negative,negative
1,1,MOVE Index,-0.021497,0.007891,-0.013175,-0.008436,0.000000,0.026303,0.000556,0.001455,...,['Mario Draghi'],['Vice Chairman Stanley Fischer'],Hausse,"{'Positive': 78, 'Negative': 75, 'Polarity': 0...","{'Positive': 40, 'Negative': 96, 'Polarity': -...",-0.411765,0.019608,neutral,negative,negative
2,2,SPX Index,-0.001872,-0.008154,0.023588,0.004086,0.003493,0.003300,0.000885,-0.011304,...,"['Benoît Cœuré', 'Benoît Cœuré']","['Vice Chairman Stanley Fischer', 'Governor La...",Hausse,"{'Positive': 92, 'Negative': 142, 'Polarity': ...","{'Positive': 114, 'Negative': 255, 'Polarity':...",-0.382114,-0.213675,negative,negative,negative
3,3,SPX Index,0.004980,-0.000864,0.001677,0.000000,0.006030,-0.001083,0.000419,0.001492,...,"['Peter Praet', 'Peter Praet']",['Governor Jerome H. Powell'],Hausse,"{'Positive': 136, 'Negative': 136, 'Polarity':...","{'Positive': 71, 'Negative': 132, 'Polarity': ...",-0.300493,0.000000,neutral,negative,negative
4,4,EURUSD Curncy,0.000360,-0.001893,0.005579,-0.003056,-0.001171,-0.001623,-0.002350,-0.006444,...,['Yves Mersch'],['Chairman Jerome H. Powell'],Hausse,"{'Positive': 32, 'Negative': 55, 'Polarity': -...","{'Positive': 69, 'Negative': 140, 'Polarity': ...",-0.339713,-0.264368,negative,negative,negative
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8941,8941,SPX Index,-0.002045,-0.003642,0.008096,0.004949,-0.001264,0.000150,0.002482,0.000000,...,['Vítor Constâncio'],['Governor Daniel K. Tarullo'],Baisse,"{'Positive': 56, 'Negative': 63, 'Polarity': -...","{'Positive': 60, 'Negative': 170, 'Polarity': ...",-0.478261,-0.058824,neutral,negative,negative
8942,8942,SX5E Index,-0.027361,0.094386,0.002350,-0.005076,0.022555,0.000963,-0.060170,0.028323,...,['Lorenzo Bini Smaghi'],"['Chairman Ben S. Bernanke', 'Governor Randall...",Baisse,"{'Positive': 84, 'Negative': 196, 'Polarity': ...","{'Positive': 100, 'Negative': 198, 'Polarity':...",-0.328859,-0.400000,negative,negative,negative
8943,8943,SX5E Index,-0.008359,0.021658,-0.015986,-0.023528,0.020072,0.017822,0.005648,0.017045,...,"['Benoît Cœuré', 'Benoît Cœuré']","['Governor Jerome H. Powell', 'Governor Lael B...",Hausse,"{'Positive': 92, 'Negative': 142, 'Polarity': ...","{'Positive': 83, 'Negative': 158, 'Polarity': ...",-0.311203,-0.213675,negative,negative,negative
8944,8944,SRVIX Index,0.011945,0.002567,0.000000,0.005115,0.000364,0.003999,0.012978,0.012458,...,"['Peter Praet', 'Vítor Constâncio']",['Vice Chair Janet L. Yellen'],Baisse,"{'Positive': 131, 'Negative': 194, 'Polarity':...","{'Positive': 127, 'Negative': 252, 'Polarity':...",-0.329815,-0.193846,negative,negative,negative


In [30]:
X = data.drop(columns=['id_series', 'Index + 1', 'list_speakers_ecb', 'list_speakers_fed', 'V_Cible_categ', 'polarity_score_ecb_LM', 'polarity_score_fed_LM', 'id_ecb', 'id_fed'])
y = data['Index + 1']
y

0       0.010597
1       0.014492
2       0.016263
3       0.013581
4       0.005092
          ...   
8941   -0.003198
8942   -0.050491
8943    0.008092
8944   -0.003784
8945    0.001246
Name: Index + 1, Length: 8946, dtype: float64

In [31]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)

In [32]:
X_train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 6709 entries, 8582 to 7270
Data columns (total 16 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   Index Name              6709 non-null   object 
 1   Index - 9               6709 non-null   float64
 2   Index - 8               6709 non-null   float64
 3   Index - 7               6709 non-null   float64
 4   Index - 6               6709 non-null   float64
 5   Index - 5               6709 non-null   float64
 6   Index - 4               6709 non-null   float64
 7   Index - 3               6709 non-null   float64
 8   Index - 2               6709 non-null   float64
 9   Index - 1               6709 non-null   float64
 10  Index - 0               6709 non-null   float64
 11  polarity_fed_LM         6709 non-null   float64
 12  polarity_ecb_LM         6709 non-null   float64
 13  sentiment_label_ecb_LM  6709 non-null   object 
 14  sentiment_label_fed_LM  6709 non-null

In [33]:
cat = CatBoostRegressor(
    iterations=500,
    learning_rate=0.01,
    cat_features=['Index Name', 'sentiment_label_ecb_LM', 'sentiment_label_fed_LM', 'sentiment_label_LM'],
    loss_function='RMSE',
    logging_level='Verbose',
    allow_writing_files=False
)
cat.fit(
    X=X_train,
    y=y_train,
    eval_set=(X_test,y_test),
    verbose = 50
)

0:	learn: 0.0414218	test: 0.0386556	best: 0.0386556 (0)	total: 8.31ms	remaining: 4.15s
50:	learn: 0.0407630	test: 0.0385664	best: 0.0385664 (50)	total: 421ms	remaining: 3.7s
100:	learn: 0.0402919	test: 0.0385084	best: 0.0385084 (100)	total: 817ms	remaining: 3.23s
150:	learn: 0.0398600	test: 0.0384628	best: 0.0384628 (150)	total: 1.25s	remaining: 2.9s
200:	learn: 0.0394615	test: 0.0384320	best: 0.0384320 (200)	total: 1.72s	remaining: 2.56s
250:	learn: 0.0391197	test: 0.0383927	best: 0.0383927 (250)	total: 2.18s	remaining: 2.16s
300:	learn: 0.0388223	test: 0.0383618	best: 0.0383618 (300)	total: 2.53s	remaining: 1.68s
350:	learn: 0.0385542	test: 0.0383547	best: 0.0383531 (347)	total: 2.91s	remaining: 1.24s
400:	learn: 0.0382383	test: 0.0383608	best: 0.0383456 (368)	total: 3.3s	remaining: 814ms
450:	learn: 0.0379671	test: 0.0383398	best: 0.0383398 (450)	total: 3.68s	remaining: 400ms
499:	learn: 0.0377312	test: 0.0383263	best: 0.0383212 (492)	total: 4.07s	remaining: 0us

bestTest = 0.038321

In [34]:
yp = cat.predict(X_test)

In [35]:
test = pd.merge(y_test, ybas, left_index=True, right_index=True)
yt = test['Index + 1'].to_numpy()
yb = test['regression'].to_numpy()
yp

array([-1.03552771e-04,  4.26452636e-05,  3.40676715e-03, ...,
        2.51023254e-03, -1.57288374e-03,  3.81689784e-04])

In [37]:
from sklearn.metrics import mean_squared_error, mean_absolute_error
print(mean_squared_error(yp, yt))
print(mean_squared_error(yb, yt))
print(mean_absolute_error(yp, yt))
print(mean_absolute_error(yb, yt))

0.0014685116442932468
0.0017258538986436834
0.021465580930181014
0.02342915025726868


In [ ]:
print('MSE : ', mean_squared_error(yp, yt))
print('MAE : ', mean_absolute_error(yb, yt))